# Exercise: Data Cleansing and Feature Engineering

In this exercise, we'll be loading in a dataset that has some problems. In order for us to get it ready for our models, we will apply some of the technics we learned.

Apply these changes to the `data.csv` dataset.
1. Load `data.csv` into a dataframe.
2. Output the table info to see if there are any null values.
3. Remove all null values from the dataframe.
4. Change the `date` column from an object to a `datetime64[ns]` type.
5. Change the `weather` column to a category type.
6. One hot encode the `date` column to year, month, and day.
7. Normalized the columns from the `all_features` list so each feature has a zero mean.
8. Create and save the cleaned dataframe, as well as the train/validation/test dataframes to CSV.

In [2]:
import random
from datetime import datetime
import pandas as pd
import numpy as np

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
# Reading the dataset created by 02_exercise_dataset_creation.ipynb
df = pd.read_csv("data.csv")

In [5]:
# Always good to check to see if the data looks right
df.head()

,feature0,feature1,feature2,date,weather,target
0,0.274647,-0.603620,0.688897,2021-01-01,sunny,41.269783
1,-0.307691,0.269024,-0.566440,2021-01-01,sunny,-147.974545
2,0.477809,-0.060138,1.974100,2021-01-01,cloudy,204.597486
3,-0.603840,-1.149554,-1.188424,2021-01-01,cloudy,-119.535892
4,0.104714,0.228053,-0.422315,2021-01-01,cloudy,-34.253007


In [6]:
# Output general info about the table, notice we have some null values in all of our features
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   feature0  997 non-null    float64
 1   feature1  985 non-null    float64
 2   feature2  991 non-null    float64
 3   date      993 non-null    object 
 4   weather   989 non-null    object 
 5   target    1000 non-null   float64
dtypes: float64(4), object(2)
memory usage: 47.0+ KB


In [7]:
# Drop all null values
df.dropna()

,feature0,feature1,feature2,date,weather,target
0,0.274647,-0.603620,0.688897,2021-01-01,sunny,41.269783
1,-0.307691,0.269024,-0.566440,2021-01-01,sunny,-147.974545
2,0.477809,-0.060138,1.974100,2021-01-01,cloudy,204.597486
3,-0.603840,-1.149554,-1.188424,2021-01-01,cloudy,-119.535892
4,0.104714,0.228053,-0.422315,2021-01-01,cloudy,-34.253007
...,...,...,...,...,...,...
995,0.936795,0.770331,0.155591,2021-03-31,cloudy,24.231439
996,-1.226196,0.183339,-0.375147,2021-03-31,rainy,-128.966545
997,0.949246,-1.502397,0.330577,2021-03-31,rainy,58.039570
998,-0.282196,-1.158203,0.283770,2021-03-31,sunny,19.150732


In [38]:
# Change the date column to a datetime
df.loc[:, 'date'] = pd.to_datetime(df['date'])
# Change weather column to a category
df.loc[:, 'weather'] = df['weather'].astype('category')

In [40]:
# Extract year, month, and day into separate columns
df['year'] = df.date.dt.year
df['month'] = df.date.dt.month
df['day'] = df.date.dt.day
df.head()

,feature0,feature1,feature2,date,weather,target,year,month,day,weather_cloudy,weather_rainy,weather_sunny
0,0.309690,-0.584488,0.738994,2021-01-01,sunny,41.269783,2021.0,1.0,1.0,-0.681270,-0.438058,0.978237
1,-0.276351,0.320039,-0.588040,2021-01-01,sunny,-147.974545,2021.0,1.0,1.0,-0.681270,-0.438058,0.978237
2,0.514143,-0.021149,2.097600,2021-01-01,cloudy,204.597486,2021.0,1.0,1.0,1.467847,-0.438058,-1.022247
3,-0.574382,-1.150368,-1.245548,2021-01-01,cloudy,-119.535892,2021.0,1.0,1.0,1.467847,-0.438058,-1.022247
4,0.138677,0.277571,-0.435683,2021-01-01,cloudy,-34.253007,2021.0,1.0,1.0,1.467847,-0.438058,-1.022247


In [41]:
# One hot encode the weather category to have individual features. Prefix with `weather`
weather_one_hot_df = pd.get_dummies(df.weather, prefix='weather')
weather_one_hot_df.head()

,weather_cloudy,weather_rainy,weather_sunny
0,0,0,1
1,0,0,1
2,1,0,0
3,1,0,0
4,1,0,0


In [17]:
# Add the one hot encoded values back to the df
df[weather_one_hot_df.columns.to_list()] = weather_one_hot_df
df.head()

,feature0,feature1,feature2,date,weather,target,year,month,day,weather_cloudy,weather_rainy,weather_sunny
0,0.274647,-0.603620,0.688897,2021-01-01,sunny,41.269783,2021.0,1.0,1.0,0,0,1
1,-0.307691,0.269024,-0.566440,2021-01-01,sunny,-147.974545,2021.0,1.0,1.0,0,0,1
2,0.477809,-0.060138,1.974100,2021-01-01,cloudy,204.597486,2021.0,1.0,1.0,1,0,0
3,-0.603840,-1.149554,-1.188424,2021-01-01,cloudy,-119.535892,2021.0,1.0,1.0,1,0,0
4,0.104714,0.228053,-0.422315,2021-01-01,cloudy,-34.253007,2021.0,1.0,1.0,1,0,0


In [18]:
# Verify now that are table info has no nulls and correct Dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   feature0        997 non-null    float64 
 1   feature1        985 non-null    float64 
 2   feature2        991 non-null    float64 
 3   date            993 non-null    category
 4   weather         989 non-null    category
 5   target          1000 non-null   float64 
 6   year            993 non-null    float64 
 7   month           993 non-null    float64 
 8   day             993 non-null    float64 
 9   weather_cloudy  1000 non-null   uint8   
 10  weather_rainy   1000 non-null   uint8   
 11  weather_sunny   1000 non-null   uint8   
dtypes: category(2), float64(7), uint8(3)
memory usage: 63.0 KB


In [21]:
# These may change if you decided to call your columns different from above
all_features = [
    "feature0",
    "feature1",
    "feature2",
    "year",
    "month",
    "day",
    "weather_cloudy",
    "weather_rainy",
    "weather_sunny",
]

In [24]:
# Table summary, notice the mean to many of our tables are not zero.
df[all_features].describe()

,feature0,feature1,feature2,year,month,day,weather_cloudy,weather_rainy,weather_sunny
count,997.000000,985.000000,991.000000,993.0,993.000000,993.000000,1000.00000,1000.000000,1000.000000
mean,-0.033086,-0.039734,-0.010171,2021.0,1.998993,15.518630,0.31700,0.161000,0.511000
std,0.994180,0.965242,0.946450,0.0,0.829763,8.712703,0.46554,0.367715,0.500129
min,-3.046143,-3.116857,-2.994613,2021.0,1.000000,1.000000,0.00000,0.000000,0.000000
25%,-0.726366,-0.737932,-0.673397,2021.0,1.000000,8.000000,0.00000,0.000000,0.000000
50%,-0.033319,-0.057043,0.010500,2021.0,2.000000,15.000000,0.00000,0.000000,1.000000
75%,0.602163,0.599040,0.656596,2021.0,3.000000,23.000000,1.00000,0.000000,1.000000
max,3.170975,2.929096,2.680571,2021.0,3.000000,31.000000,1.00000,1.000000,1.000000


In [25]:
# Standarize feature values to have a zero mean
scaler = StandardScaler()
scaler.fit(df[all_features])
df.loc[:, all_features] = scaler.transform(df[all_features])

In [26]:
# Verify our features we are using now all have zero mean
df[all_features].describe()

,feature0,feature1,feature2,year,month,day,weather_cloudy,weather_rainy,weather_sunny
count,9.970000e+02,9.850000e+02,9.910000e+02,993.0,9.930000e+02,9.930000e+02,1.000000e+03,1.000000e+03,1.000000e+03
mean,-5.584522e-17,-2.028834e-18,-1.915723e-17,0.0,1.914101e-16,6.703824e-16,-1.587619e-16,-7.771561e-19,-3.907985e-16
std,1.000502e+00,1.000508e+00,1.000505e+00,0.0,1.000504e+00,1.000504e+00,1.000500e+00,1.000500e+00,1.000500e+00
min,-3.032215e+00,-3.189548e+00,-3.154895e+00,0.0,-1.204556e+00,-1.667215e+00,-6.812700e-01,-4.380583e-01,-1.022247e+00
25%,-6.976878e-01,-7.237070e-01,-7.011052e-01,0.0,-1.204556e+00,-8.633852e-01,-6.812700e-01,-4.380583e-01,-1.022247e+00
50%,-2.344522e-04,-1.794117e-02,2.185140e-02,0.0,1.214270e-03,-5.955577e-02,-6.812700e-01,-4.380583e-01,9.782368e-01
75%,6.392891e-01,6.621118e-01,7.048485e-01,0.0,1.206985e+00,8.591065e-01,1.467847e+00,-4.380583e-01,9.782368e-01
max,3.224434e+00,3.077299e+00,2.844420e+00,0.0,1.206985e+00,1.777769e+00,1.467847e+00,2.282801e+00,9.782368e-01


In [27]:
# train: 0.8 | test: 0.2
df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)

# train: 0.6 | validation: 0.2
df_train, df_val = train_test_split(df_train, test_size=0.25, random_state=0)

# Final dataset sizes: train: 0.6, validation: 0.2, text: 0.2,

In [28]:
# Output each shape to confirm the size of train/validation/test
print(f"Train: {df_train.shape}")
print(f"Validation: {df_val.shape}")
print(f"Test: {df_test.shape}")

Train: (600, 12)
Validation: (200, 12)
Test: (200, 12)


In [31]:
# Save all clean data, and the train, validation, test data as csv
df.to_csv('cleaned_data.csv', index=False)
df_train.to_csv('train.csv', index=False)
df_val.to_csv('validation.csv', index=False)
df_test.to_csv('test.csv', index=False)